In [1]:
# pip install world_bank_data --upgrade

In [ ]:
import pandas as pd
import world_bank_data as wb
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Documentation
https://pypi.org/project/world-bank-data/

Example: https://mybinder.org/v2/gh/mwouts/world_bank_data/master

https://hub.gke2.mybinder.org/user/mwouts-world_bank_data-64c2qdnf/tree/examples

### Overview API commands for testing purposes

In [ ]:
#API commands
wb.get_topics()
wb.get_sources()
wb.get_countries()
wb.get_indicators(topics=1)

### Overview of Indicators we are interested in

* SP.POP.TOTL     : population
* NY.GDP.MKTP.CD  :GDP
* NY.GDP.PCAP.CD  :GDP_per_cap
* SP.DYN.LE00.IN  :Life expectancy
* SH.STA.DIAB.ZS  :Diabetes percent (% of population ages 20 to 79)
* EN.ATM.CO2E.PC  :CO2 per cap
* SL.UEM.TOTL.ZS  :Unemployment
* EG.USE.PCAP.KG.OE :energy use per cap ## 224 empty
* SE.PRM.CMPT.ZS : primary schooling competion rate # viel leer
* SE.PRM.ENRL.TC.ZS   ::Pupil-teacher ratio, primary # 213 nan
* SE.ADT.1524.LT.ZS :: literacy_youth #viel leer
* SE.XPD.TOTL.GD.ZS :: Government expenditure on education, total (% of GDP)#203 leer
* SP.RUR.TOTL.ZS ::: % Rural Pop
* EG.USE.PCAP.KG.OE :: Energy use per capita in kg of oil
* EG.FEC.RNEW.ZS Renewable energy consumption (% of total final energy consumption)
* EN.ATM.PM25.MC.ZS Population suffering fom airpolution


### Setting up our dataframe with countries and indicators


In [ ]:
#picking countries
df=pd.DataFrame()
df['Country']=list(wb.get_countries()['name'].loc[wb.get_countries()['region'] != 'Aggregates'])

In [ ]:
indicators = ["SP.POP.TOTL","NY.GDP.MKTP.CD","NY.GDP.PCAP.CD",'SP.DYN.LE00.IN','SH.STA.DIAB.ZS','EN.ATM.CO2E.PC','SE.PRM.ENRL.TC.ZS','SL.UEM.TOTL.ZS','SE.XPD.TOTL.GD.ZS','SP.RUR.TOTL.ZS','EG.FEC.RNEW.ZS','EN.ATM.PM25.MC.ZS']
from functools import reduce
data_all = []
for i in range(0, (len(indicators))):
    data = pd.DataFrame(wb.get_series(indicators[i],mrv=1)).reset_index()
    data = data.drop(labels=['Series','Year'],axis='columns')
    data_all.append(data)
    
data_all
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='Country'), data_all).rename(columns={"SP.POP.TOTL":'population',"NY.GDP.MKTP.CD":'GDP_in_Dollar','NY.GDP.PCAP.CD':'gdp_per_cap','SP.DYN.LE00.IN':'life_exp','SH.STA.DIAB.ZS':'diabetes_percent','EN.ATM.CO2E.PC':'co2_per_cap','SL.UEM.TOTL.ZS':'unemp','SE.PRM.ENRL.TC.ZS':'teacher','SP.RUR.TOTL.ZS':'rural_pop','EG.FEC.RNEW.ZS':'renewable_percent','SE.XPD.TOTL.GD.ZS':'educat_expen','EN.ATM.PM25.MC.ZS':'airpollution'})
df

In [ ]:
# all features with explanation
descriptions=[]
for i in range(len(indicators)):
    descriptions.append(wb.get_series(indicators[i],mrv=1).reset_index()['Series'].unique())

desc=pd.DataFrame(descriptions,indicators).reset_index().rename(columns={'index':'abbrev','0':'Description'})

In [ ]:
desc

#### addin a line indicating how many values are missing in the respective column

In [ ]:
empty=pd.DataFrame(df.isnull().sum()).transpose()

In [ ]:
pd.concat([empty,df],axis=0)

### droping columns with loads of missing data

In [ ]:
df.drop(["teacher","educat_expen"], axis=1, inplace=True)

### Subsetting for countries that miss no data

In [ ]:
df.dropna(axis=0, how='any',inplace=True)
df.head()

### LOOKING AT INDIVIFUAL VALUES

### multicollinearity

In [ ]:
df_corr = pd.DataFrame(df,columns=['Population_in_mil','gdp_per_cap',"life_exp","diabetes_percent","co2_per_cap","unemp","rural_pop","renewable_percent","airpolution"]).corr()
df_corr

In [ ]:
corr = df.drop(['GDP_in_Dollar','population'],axis=1).corr() #correlation matix ohne gsd und ohne population
f, ax = plt.subplots(figsize=(15, 15)) 
mask = np.triu(np.ones_like(corr, dtype=bool)) # removing the other side of the heatmap 
cmap = sns.diverging_palette(230, 20, as_cmap=True) 
#preparing cmap 
sns.heatmap(corr,mask=mask,cmap=cmap,linewidths=.5,square=True,annot=True) 
plt.show()

### Standardizing numeric features

In [ ]:
# standardizer
from sklearn.preprocessing import StandardScaler
df_num=df.select_dtypes(include = np.number)
transformer = StandardScaler().fit(df_num)
num_stand_array = transformer.transform(df_num)
df_num_stand=pd.DataFrame(num_stand_array, columns=df_num.columns)
df_num_stand.describe()
#looking good

In [ ]:
df_num_stand

### How many clusters we need with Silhouette Score

In [ ]:
df_num_stand.drop(['GDP_in_Dollar','population'],axis=1,inplace=True)

In [ ]:
# How many clusters???
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

K = range(2,12) # 
silhouette_kmeans = []

for k in K:
    model = KMeans(n_clusters=k)
    model.fit(df_num_stand)
    silhouette_kmeans.append(silhouette_score(df_num_stand, model.fit_predict(df_num_stand)))

plt.figure(figsize=(16,8))
sns.set_style('darkgrid')
plt.plot(K, silhouette_kmeans, 'bo-',color='black')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Score for KMeans')

### KMEANS WITH 3 CLUSTERS

In [ ]:
#3 cluster solution
from sklearn.cluster import KMeans

#labels
model_3 = KMeans(n_clusters=3, random_state = 1000).fit(df_num_stand)

df_num_stand["cluster"] = model_3.labels_ #adding a column that tells in which cluster a record ends up
df_num_stand

In [ ]:
df_cluster_3=pd.concat([df.reset_index(drop=True),df_num_stand.cluster.reset_index(drop=True)],axis=1)
df_cluster_3

In [ ]:
df_cluster_3['cluster'].value_counts()

### KMEANS MAP PRESENTATION 3 CLUSTERS
GEO PANDAS https://geopandas.org/mapping.html

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world=world.rename(columns={'name':'Country'})

various countries are slightly differently named by geopandas vs. worldbank.
* trying to take care of this with manually setting up a dictionary
* countries will be missing in the map - noteably France & Italy - since they had missing values in the releveant world bank data

In [ ]:
dic={'United States of America':'United States',
'Russia':'Russian Federation',
'South Korea': 'Korea, Rep.',
'Egypt': 'Egypt, Arab Rep.',
'Bosnia and Herz.':'Bosnia and Herzegovina',
'Central African Rep.':'Central African Republic',
"Côte d'Ivoire":"Cote d'Ivoire",
'Dem. Rep. Congo':'Congo, Dem. Rep.',
'Eq. Guinea':'Equatorial Guinea',
'Yemen':'Yemen, Rep.',
'Slovakia':'Slovak Republic',
'Kyrgyzstan':'Kyrgyz Republic'}

In [ ]:
world.Country=world.Country.replace(dic)

In [ ]:
world_3=world.merge(df_cluster_3[['Country','cluster']], on='Country', how='left')
world_3['cluster']=world_3['cluster']


In [ ]:

fig, ax = plt.subplots(figsize=(20,15))
world_3.plot(ax=ax, column='cluster',cmap='tab20b')

ax.set(xlabel='Longitude(Degrees)',ylabel='Latitude(Degrees)',title='3 CLUSTER MAP')
plt.show()

###  KMEANS 3 CLUSTERS GDP vs DIABETES


In [ ]:
plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=df_cluster_3,x='gdp_per_cap',y='diabetes_percent',style='cluster', hue='cluster', palette='pastel',s=200)


for i in range (len(df_cluster_3)):
    ax.annotate(df_cluster_3['Country'].iloc[i], (df_cluster_3['gdp_per_cap'].iloc[i],df_cluster_3['diabetes_percent'].iloc[i]))
plt.show()

### KMEANS 3 CLUSTERS CO2 Emission vs Renwable Energy %

In [ ]:
plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=df_cluster_3,x='renewable_percent',y='co2_per_cap',style='cluster', hue='cluster', palette='pastel',s=200)


for i in range (len(df_cluster_3)):
    ax.annotate(df_cluster_3['Country'].iloc[i], (df_cluster_3['renewable_percent'].iloc[i],df_cluster_3['co2_per_cap'].iloc[i]))
plt.show()

### KMEANS WITH 5 CLUSTERS

In [ ]:
from sklearn.cluster import KMeans

#labels
model_5 = KMeans(n_clusters=5, random_state = 1000).fit(df_num_stand)

df_num_stand["cluster"] = model_5.labels_ #adding a column that tells in which cluster a record ends up
df_num_stand

In [ ]:
# ADDING CLUSTER TO THE ORIGINAL DATAFRAME WITH COUNTRY NAMES AGAIN
df_cluster_5=pd.concat([df.reset_index(drop=True),df_num_stand.cluster.reset_index(drop=True)],axis=1)
df_cluster_5

In [ ]:
df_cluster_5['cluster'].value_counts()

### KMEANS MAP PRESENTATION 5 CLUSTERS

In [ ]:
world_5=world.merge(df_cluster_5[['Country','cluster']], on='Country', how='left')
world_5['cluster']=world_5['cluster']

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
world_5.plot(ax=ax, column='cluster',cmap='tab20b',label='abc')
ax.set(xlabel='Longitude(Degrees)',ylabel='Latitude(Degrees)',title='5 CLUSTER MAP')


plt.show()

In [ ]:
plt.figure(figsize=(10,10))
ax=sns.scatterplot(data=df_cluster_5,x='gdp_per_cap',y='diabetes_percent',style='cluster', hue='cluster', palette='pastel',s=400)


for i in range (len(df_cluster_5)):
    ax.annotate(df_cluster_5['Country'].iloc[i], (df_cluster_5['gdp_per_cap'].iloc[i],df_cluster_5['diabetes_percent'].iloc[i]))

# Hierarchical Clustering (Agglomerative)
DIDNT MAKE IT WORK

In [ ]:
# starting point my standardized dataframe
df_agg=df_num_stand.drop(['cluster'],axis=1)


In [ ]:
from sklearn.cluster import AgglomerativeClustering #is hierarchial clustering

In [ ]:
K = range(2, 8)#different number of clusters
silhouette = []

for k in K:
    model = AgglomerativeClustering(n_clusters=k)
    model.fit(df_agg)
    silhouette.append(silhouette_score(df_agg, model.fit_predict(df_agg)))

plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Score for Agglomerative Clustering')

# only wants 2 clusters

In [ ]:
model_agg = AgglomerativeClustering(n_clusters=2)# chose number of clusters by n_clusters, distance_thershold would be a different option
df_agg['cluster'] = model_agg.fit_predict(df_agg)
df_agg.head()

In [ ]:
# ADDING CLUSTER TO THE ORIGINAL DATAFRAME WITH COUNTRY NAMES AGAIN
df_agg_2=pd.concat([df.reset_index(drop=True),df_agg.cluster.reset_index(drop=True)],axis=1)
df_agg_2['cluster'].value_counts()

In [ ]:
df_agg_2

In [ ]:
#dendrogram
from scipy.cluster.hierarchy import dendrogram, linkage
Z = linkage(df_agg)
plt.figure(figsize=(25, 10))
dendrogram(Z) 
#plt.xticks(np.arange(len(df_agg_2)),list(df_agg_2.Country))
plt.show()

In [ ]:
#######################DIDNT MAKE IT WORK#######################

# PCA


In [ ]:
#overviewing our features
df.columns

In [ ]:
from sklearn.decomposition import PCA

X=df.drop(['Country','population', 'gdp_per_cap'],axis=1)
#gdp_per_cap important!

pca=PCA(n_components=4)
pca.fit(X)
components=pca.transform(X)
pd.DataFrame(components)


# KMEANS CLUSTERIN WITHOUT ANY GDP VALUE

In [ ]:
#picking countries
df=pd.DataFrame()
df['Country']=list(wb.get_countries()['name'].loc[wb.get_countries()['region'] != 'Aggregates'])

In [ ]:
indicators = ["SP.POP.TOTL","NY.GDP.MKTP.CD","NY.GDP.PCAP.CD",'SP.DYN.LE00.IN','SH.STA.DIAB.ZS','EN.ATM.CO2E.PC','SE.PRM.ENRL.TC.ZS','SL.UEM.TOTL.ZS','SE.XPD.TOTL.GD.ZS','SP.RUR.TOTL.ZS','EG.FEC.RNEW.ZS','EN.ATM.PM25.MC.ZS']
from functools import reduce
data_all = []
for i in range(0, (len(indicators))):
    data = pd.DataFrame(wb.get_series(indicators[i],mrv=1)).reset_index()
    data = data.drop(labels=['Series','Year'],axis='columns')
    data_all.append(data)
    
data_all
df = reduce(lambda df1,df2: pd.merge(df1,df2,on='Country'), data_all).rename(columns={"SP.POP.TOTL":'population',"NY.GDP.MKTP.CD":'GDP_in_Dollar','NY.GDP.PCAP.CD':'gdp_per_cap','SP.DYN.LE00.IN':'life_exp','SH.STA.DIAB.ZS':'diabetes_percent','EN.ATM.CO2E.PC':'co2_per_cap','SL.UEM.TOTL.ZS':'unemp','SE.PRM.ENRL.TC.ZS':'teacher','SP.RUR.TOTL.ZS':'rural_pop','EG.FEC.RNEW.ZS':'renewable_percent','SE.XPD.TOTL.GD.ZS':'educat_expen','EN.ATM.PM25.MC.ZS':'airpollution'})


In [ ]:
df.drop(["teacher","educat_expen",'population','gdp_per_cap','GDP_in_Dollar'], axis=1, inplace=True)
df

In [ ]:
df.dropna(axis=0, how='any',inplace=True)
df.head()

#### Correlation Matrix WITHOUT ANY GDP VALUES

In [ ]:
corr = df.corr() #correlation matix ohne gsd und ohne population
f, ax = plt.subplots(figsize=(15, 15)) 
mask = np.triu(np.ones_like(corr, dtype=bool)) # removing the other side of the heatmap 
cmap = sns.diverging_palette(230, 20, as_cmap=True) 
#preparing cmap 
sns.heatmap(corr,mask=mask,cmap=cmap,linewidths=.5,square=True,annot=True) 
plt.show()

In [ ]:
# standardizer
from sklearn.preprocessing import StandardScaler
df_num=df.select_dtypes(include = np.number)
transformer = StandardScaler().fit(df_num)
num_stand_array = transformer.transform(df_num)
df_num_stand=pd.DataFrame(num_stand_array, columns=df_num.columns)
df_num_stand.describe()


#### SILOUHETTE SCORE WITHOUT ANY GDP VALUES

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

K = range(2,12) # 
silhouette_kmeans = []

for k in K:
    model = KMeans(n_clusters=k)
    model.fit(df_num_stand)
    silhouette_kmeans.append(silhouette_score(df_num_stand, model.fit_predict(df_num_stand)))

plt.figure(figsize=(16,8))
sns.set_style('darkgrid')
plt.plot(K, silhouette_kmeans, 'bo-',color='black')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Score for KMeans')

####
# 5 clusters looks best

In [ ]:
#5 cluster solution
from sklearn.cluster import KMeans

#labels
model_5_nogdp= KMeans(n_clusters=5, random_state = 1000).fit(df_num_stand)

df_num_stand["cluster"] = model_5_nogdp.labels_ #adding a column that tells in which cluster a record ends up
df_num_stand

In [ ]:
df_cluster_5_nogdp=pd.concat([df.reset_index(drop=True),df_num_stand.cluster.reset_index(drop=True)],axis=1)


In [ ]:
df_cluster_5_nogdp['cluster'].value_counts()

#### MAPPING WITHOUT ANY GDP VALUES

In [ ]:
world_5_nogdp=world.merge(df_cluster_5_nogdp[['Country','cluster']], on='Country', how='left')
world_5_nogdp['cluster']=world_5_nogdp['cluster']

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
world_5_nogdp.plot(ax=ax, column='cluster',cmap='tab20',label='abc')
ax.set(xlabel='Longitude(Degrees)',ylabel='Latitude(Degrees)',title='5 CLUSTER MAP WITHOUT GDP VALUES')


plt.show()